<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.4: Functional Programming 函数式编程
**Prev: [Higher-Order Functions 高层次函数](3.3_higher-order_functions.ipynb)**<br>
**Next: [Object Oriented Programming 面向对象编程](3.5_object_oriented_programming.ipynb)**

## Motivation 动机
You saw functions in many previous modules, but now it's time to make our own and use them effectively. 你在前面章节中看到了很多的函数式，不过现在我们应该自己尝试编写并且有效地使用了。

## Setup 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

This module uses the Chisel `FixedPoint` type, which currently resides in the experimental package. 这个单元使用了 Chisel 类型 `FixedPoint`，现在还只是在实验性函数包中。

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import chisel3.experimental._
import chisel3.internal.firrtl.KnownBinaryPoint

---
# Functional Programming in Scala       Scala 中的函数式编程
Scala functions were introduced in Module 1, and you saw then used a lot in the previous module. Here's a refresher on functions. Functions take any number of inputs and produce one output. Inputs are often called arguments to a function. To produce no output, return the `Unit` type.     Scala 函数在第一个单元中就已经被介绍过了，在前面的单元中你也见到了不少例子。现在是函数的复习。函数是接收一定数量的输入，并且产生一个输出。输入通常被称作是一个函数的参数。如果不产生任何输出，将会返回 `Unit` 类型。

<span style="color:blue">**Example: Custom Functions 举例：自定义函数**</span><br>
Below are some examples of functions in Scala. 下面有一些 Scala 函数的例子。

In [ ]:
// No inputs or outputs (two versions). 没有输入或者输出 (两个版本)
def hello1(): Unit = print("Hello! 你好！")
def hello2 = print("Hello again! 再一次问好！")

// Math operation: one input and one output. 数学运算：一个输入一个输出
def times2(x: Int): Int = 2 * x

// Inputs can have default values, and explicitly specifying the return type is optional. 输入可以有一些默认值，然后显示地声明返回值的类型是可选的。
// Note that we recommend specifying the return types to avoid surprises/bugs. 注意，我们建议显示地声明返回值来避免一些故障。
def timesN(x: Int, n: Int = 2) = n * x

// Call the functions listed above. 调用上面的函数。
hello1()
hello2
times2(4)
timesN(4)         // no need to specify n to use the default value 不需要给 n 赋值，使用默认值就行
timesN(4, 3)      // argument order is the same as the order where the function was defined 参数的顺序与在函数中定义的顺序相同。
timesN(n=7, x=2)  // arguments may be reordered and assigned to explicitly 参数也可以显示地乱序赋值。

## Functions as Objects 函数对象
Functions in Scala are first-class objects. That means we can assign a function to a `val` and pass it to classes, objects, or other functions as an argument. 在 Scala 中，函数是第一类的对象。这意味着我们可以通过 `val` 把一个函数赋值给变量，并且将它作为参数传递给类、对象或者其他函数

<span style="color:blue">**Example: Function Objects 举例：函数对象**</span><br>
Below are the same functions implemented as functions and as objects. 下面是分别使用函数式和对象实现的函数。

In [ ]:
// These are normal functions. 这是普通的函数
def plus1funct(x: Int): Int = x + 1
def times2funct(x: Int): Int = x * 2

// These are functions as vals. 这些函数是变量
// The first one explicitly specifies the return type. 第一个显示地声明了返回值的类型。
val plus1val: Int => Int = x => x + 1
val times2val = (x: Int) => x * 2

// Calling both looks the same. 调用它们的时候看起来一样。
plus1funct(4)
plus1val(4)
plus1funct(x=4)
//plus1val(x=4) // this doesn't work 这不行

Why would you want to create a `val` instead of a `def`? With a `val`, you can now pass the function around to other functions, as shown below. You can even create your own functions that accept other functions as arguments. Formally, functions that take or produce functions are called *higher-order functions*. You saw them used in the last module, but now you'll make your own! 为什么你想声明一个 `val` 而不是 `def` 呢？使用 `val`，你可以把一个函数传递给另外一个，就像下面这样。你创建的自己的函数，甚至可以接收其他函数作为变量。用术语来说，能够产生或者接收函数的函数被称作*高阶函数*。在上一个单元中你看到了他们，现在是时候自己写了！

<span style="color:blue">**Example: Higher-Order Functions 举例：高阶函数**</span><br>
Here we show `map` again, and we also create a new function, `opN`, that accepts a function, `op`, as an argument. 我们再一次展示 `map`，我们也创造一个新的函数 `opN`，这个函数接收另外一个函数作为它的参数。

In [ ]:
// create our function 声明我们的函数
val plus1 = (x: Int) => x + 1
val times2 = (x: Int) => x * 2

// pass it to map, a list function 把它传递给 map 函数，这是一个列表函数
val myList = List(1, 2, 5, 9)
val myListPlus = myList.map(plus1)
val myListTimes = myList.map(times2)

// create a custom function, which performs an operation on X N times using recursion 声明一个自定义的函数，这个函数使用递归来实现对 X 操作 N 次。
def opN(x: Int, n: Int, op: Int => Int): Int = {
  if (n <= 0) { x }
  else { opN(op(x), n-1, op) }
}

opN(7, 3, plus1)
opN(7, 3, times2)

<span style="color:blue">**Example: Functions vs. Objects 举例：函数与对象**</span><br>
A possibly confusing situation arises when using functions without arguments. Functions are evaluated every time they are called, while `val`s are evaluated at instantiation. 当我们使用不带参数的函数时，我们可能会疑惑。每一次函数被调用时都会被计算，但 `val` 只有在声明的时候才被计算。

In [ ]:
import scala.util.Random

// both x and y call the nextInt function, but x is evaluated immediately and y is a function    x 和 y 都调用了 nextInt 函数，但是 x 立即就被计算了，y 是一个函数。
val x = Random.nextInt
def y = Random.nextInt

// x was previously evaluated, so it is a constant 因为 x 在前面就已经被计算了，所以它现在是一个常数。
println(s"x = $x")
println(s"x = $x")

// y is a function and gets reevaluated at each call, thus these produce different results    y是一个函数，在每一次被调用的时候都会重新计算一下，所以下面的调用会产生不一样的结果。
println(s"y = $y")
println(s"y = $y")

## Anonymous Functions 匿名函数
As the name implies, anonymous functions are nameless. There's no need to create a `val` for a function if we'll only use it once. 匿名函数是没有名字的。如果某个函数是一次性的话，我们就没有必要使用 `val` 给它取一个名字。

<span style="color:blue">**Example: Anonymous Functions 举例：匿名函数**</span><br>
The following example demonstrates this. They are often scoped (put in curly braces instead of parentheses). 下面的例子解释了匿名函数。它们通常是被限定的，放在尖括号内而不是圆括号。

In [ ]:
val myList = List(5, 6, 7, 8)

// add one to every item in the list using an anonymous function 使用匿名函数来给每一个列表里面的元素加一
// arguments get passed to the underscore variable 参数通过下划线变量传递
// these all do the same thing 他们实现的功能是一样的
myList.map( (x:Int) => x + 1 )
myList.map(_ + 1)

// a common situation is to use case statements within an anonymous function 一个通常的使用情景
val myAnyList = List(1, 2, "3", 4L, myList)
myAnyList.map {
  case (_:Int|_:Long) => "Number"
  case _:String => "String"
  case _ => "error"
}

<span style="color:red">**Exercise: Sequence Manipulation 举例：序列操作**</span><br>
A common set of higher-order functions you'll use are `scanLeft`/`scanRight`, `reduceLeft`/`reduceRight`, and `foldLeft`/`foldRight`. It's important to understand how each one works and when to use them. The default directions for `scan`, `reduce`, and `fold` are left, though this is not guaranteed for all cases. 一组你即将看到的高阶函数是 `scanLeft`/`scanRight`, `reduceLeft`/`reduceRight` 和 `foldLeft`/`foldRight`。理解每一个怎样工作并且什么时候使用他们是很重要的。`scan`、`reduce` 和 `fold` 的默认方向是向左，尽管这不能保证所有情况都这样。

In [ ]:
val exList = List(1, 5, 7, 100)

// write a custom function to add two numbers, then use reduce to find the sum of all values in exList 编写一个把两个数相加的函数，然后使用 `reduce` 得到在 `exList` 中所有值的和。
def add(a: Int, b: Int): Int = ???
val sum = ???

// find the sum of exList using an anonymous function (hint: you've seen this before!) 使用匿名函数得到 `exList` 中所有值的和。 (提示：你已经在之前看到过了！)
val anon_sum = ???

// find the moving average of exList from right to left using scan; make the result (ma2) a list of doubles 使用 `scan` 从右到左找到 `exList` 的移动平均值；使得结果 (ma2) 是双精度列表。
def avg(a: Int, b: Double): Double = ???
val ma2 = ???

In [ ]:
assert(add(88, 88) == 176)
assert(sum == 113)

assert(anon_sum == 113)

assert(avg(100, 100.0) == 100.0)
assert(ma2 == List(8.875, 16.75, 28.5, 50.0, 0.0))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution 解答</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def add(a: Int, b: Int): Int = a + b
val sum = exList.reduce(add)

val anon\_sum = exList.reduce(\_ + \_)

def avg(a: Int, b: Double): Double = (a + b)/2.0
val ma2 = exList.scanRight(0.0)(avg)
</pre></article></div></section></div>

---
# Functional Programming in Chisel 在 Chisel 中的函数式编程
Let's look at some examples of how to use functional programming when creating hardware generators in Chisel. 让我们来看看在 Chisel 中声明硬件生成器的时候怎么使用函数式编程。

<span style="color:blue">**Example: FIR Filter 举例：FIR 滤波器**</span><br>
First, we'll revisit the FIR filter from previous examples. Instead of passing in the coefficients as parameters to the class or making them programmable, we'll pass a function to the FIR that defines how the window coefficients are calculated. This function will take the window length and bitwidth to produce a scaled list of coefficients. Here are two example windows. To avoid fractions, we'll scale the coefficients to be between the max and min integer values. For more on these windows, check out the [this Wikipedia page](https://en.wikipedia.org/wiki/Window_function). 首先，我们会重新审视之前的 FIR 滤波器的例子。我们这一次不是通过把系数当中类的变量传递或者是让它们变得可编程化，而是把一个函数传递进去。这个函数定义了窗口系数是怎么计算的。这个函数将会接受出口长度和宽度，并且产生一个成比例的系数列表。下面有两种窗口。为了避免分数，我们将把系数缩放为最大和最小的整数值之间的值。更多的窗口函数，请参照[维基百科](https://en.wikipedia.org/wiki/Window_function).

In [ ]:
// get some math functions 导入一些数学函数
import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window 简单的三角形窗口
val TriangularWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x:Int) => 1-abs((x.toDouble-(length-1)/2.0)/((length-1)/2.0)) )
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// Hamming window 海明窗
val HammingWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x: Int) => 0.54 - 0.46*cos(2*Pi*x/(length-1)))
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// check it out! first argument is the window length, and second argument is the bitwidth 检查一下，第一个参数是窗口的长度，第二个参数是比特宽度
TriangularWindow(10, 16)
HammingWindow(10, 16)

Now we'll create a FIR filter that accepts a window function as the argument. This allows us to define new windows later on and retain the same FIR generator. It also allows us to independently size the FIR, knowing the window will be recalculated for different lengths or bitwidths. Since we are choosing the window at compile time, these coefficients are fixed. 现在我们将会声明一个接收窗口函数作为参数的 FIR 滤波器。这允许我们在后面定义一个新的窗口函数但是保留相同的 FIR 滤波器。这也使得我们可以独立地限定 FIR 的大小，因为窗口会根据长度和宽度重新计算。我们在编译的时候才选择窗口函数，窗口函数的系数是固定的。

In [ ]:
// our FIR has parameterized window length, IO bitwidth, and windowing function 我们的 FIR 有参数化的窗口长度、IO 宽度、窗口函数
class MyFir(length: Int, bitwidth: Int, window: (Int, Int) => Seq[Int]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt((bitwidth*2+length-1).W)) // expect bit growth, conservative but lazy 
  })

  // calculate the coefficients using the provided window function, mapping to UInts 使用提供的窗口函数计算对应的系数，映射到无符号整数。
  val coeffs = window(length, bitwidth).map(_.U)
  
  // create an array holding the output of the delays 声明一个数组，保留延迟的输出。
  // note: we avoid using a Vec here since we don't need dynamic indexing 注意：我们避免在这里使用 Vec，因为我们不需要动态索引
  val delays = Seq.fill(length)(Wire(UInt(bitwidth.W))).scan(io.in)( (prev: UInt, next: UInt) => {
    next := RegNext(prev)
    next
  })
  
  // multiply, putting result in "mults" 相乘，把结果放进"mults" 
  val mults = delays.zip(coeffs).map{ case(delay: UInt, coeff: UInt) => delay * coeff }
  
  // add up multiplier outputs with bit growth 把上一步得到的积累加起来
  val result = mults.reduce(_+&_)

  // connect output 把结果与输出相连
  io.out := result
}

Those last three lines could be easily combined into one. Also notice how we've handled bitwidth growth conservatively to avoid loss. 上面的最后三行可以用一行表示。同时也注意，我们已经比较保守地控制了比特宽度的增长以避免损失。

<span style="color:blue">**Example: FIR Filter Tester 举例：FIR 滤波器测试**</span><br>
Let's test our FIR! Previously, we provided a custom golden model. This time we'll use Breeze, a Scala library of useful linear algebra and signal processing functions, as a golden model for our FIR filter. The code below compares the Chisel output with the golden model output, and any errors cause the tester to fail. 让我们来测试滤波器。早些时候，我们提供了一个自定义的黄金模型。这一次，我们将会使用 Breeze 这个 Scala 库。这个库提供了很多线性代数和信号处理函数，所以我们可以把它用作我们的黄金模型。下面的这段代码比较了 Chisel 的输出与黄金模型的输出，有任何一个错误都会导致测试失败。

Try uncommenting the print statment at the end just after the expect call. Also try changing the window from triangular to Hamming. 尝试把最后的打印语句取消注释。同时也尝试把窗口函数从三角函数变到海明窗口。

In [ ]:
// math imports 导入数学库
import scala.math.{pow, sin, Pi}
import breeze.signal.{filter, OptOverhang}
import breeze.signal.support.{CanFilter, FIRKernel1D}
import breeze.linalg.DenseVector

// test parameters 测试变量
val length = 7
val bitwidth = 12 // must be less than 15, otherwise Int can't represent the data, need BigInt 必须比 15 少，不然 Int 不能代表这个数据，应该使用 BigInt 类型
val window = TriangularWindow

// test our FIR 测试我们的 FIR 滤波器
Driver(() => new MyFir(length, bitwidth, window)) {
  c => new PeekPokeTester(c) {
    
    // test data
    val n = 100 // input length 输入长度
    val sine_freq = 10
    val samp_freq = 100
    
    // sample data, scale to between 0 and 2^bitwidth 数据样本，大小在 0 到 2^bitwidth 之间
    val max_value = pow(2, bitwidth)-1
    val sine = (0 until n).map(i => (max_value/2 + max_value/2*sin(2*Pi*sine_freq/samp_freq*i)).toInt)
    //println(s"input = ${sine.toArray.deep.mkString(", ")}")
    
    // coefficients 系数
    val coeffs = window(length, bitwidth)
    //println(s"coeffs = ${coeffs.toArray.deep.mkString(", ")}")

    // use breeze filter as golden model; need to reverse coefficients
    val expected = filter(DenseVector(sine.toArray), 
                          FIRKernel1D(DenseVector(coeffs.reverse.toArray), 1.0, ""), 
                          OptOverhang.None)
    //println(s"exp_out = ${expected.toArray.deep.mkString(", ")}")

    // push data through our FIR and check the result 把数据输入到我们的滤波器中并且检查结果
    reset(5)
    for (i <- 0 until n) {
      poke(c.io.in, sine(i))
      if (i >= length-1) { // wait for all registers to be initialized since we didn't zero-pad the data 等待所有寄存器初始化，因为我们没有给数据前面加零
        expect(c.io.out, expected(i-length+1))
        //println(s"cycle $i, got ${peek(c.io.out)}, expect ${expected(i-length+1)}")
      }
      step(1)
    }
  }
}

---
# Chisel Exercises Chisel 练习
Complete the following exercises to practice writing functions, using them as arguments to hardware generators, and avoiding mutable data. 完成下面的练习以熟悉函数，把他们当作参数传进硬件生成器中，并且避免不定的数据。

<span style="color:red">**Exercise: Neural Network Neuron 练习：神经网络神经元**</span><br>
Our first example will have you build a neuron, the building block of fully-connected layers in artificial neural networks. Neurons take inputs and a set of weights, one per input, and produce one output. The weights and inputs are multiplied and added, and the result is fed through an activation function. In this exercise, you will implement different activation functions and pass them as an argument to your neuron generator. 我们第一个例子将会让你建立一个神经元，也即是组成人工神经网络中的全连接层的基本组成模块。神经元接收输入和每一个输入对应的一组权重，然后产生一个输出。权重和输入会被相乘和相加，并且把结果通过一个激活函数。在这个练习中，你将会实现不同激活函数并且把他们当作参数传递到你的神经元生成器中。

![Neuron 神经元](https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/ArtificialNeuronModel_english.png/600px-ArtificialNeuronModel_english.png)

First, complete the following code to create a neuron generator. The parameter `inputs` gives the number of inputs. The parameter `act` is a function that implements the logic of the activation function. We'll make the inputs and outputs 16-bit fixed point values with 8 fractional bits. 首先，完成下面的这段神经元生成器代码。 `inputs` 提供了输入的数量。`act` 是实现了激活函数逻辑的函数。我们的输入输出都将是 16 位固定小数点、8 位小数的值。

In [ ]:
class Neuron(inputs: Int, act: FixedPoint => FixedPoint) extends Module {
  val io = IO(new Bundle {
    val in      = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val weights = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val out     = Output(FixedPoint(16.W, 8.BP))
  })
  
  ???
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution 解答</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val mac = io.in.zip(io.weights).map{ case(a:FixedPoint, b:FixedPoint) => a*b}.reduce(_+_)
  io.out := act(mac)
</pre></article></div></section></div>

Now let's create some activation functions! We'll use a threshold of zero. Typical activation functions are the sigmoid function and the rectified linear unit (ReLU). 现在让我们声明一些激活函数！我们将使用零作为阈值。通常激活函数是 S 型的函数和整流线形单元 (ReLU)。

The sigmoid we'll use is called the [logistic function](https://en.wikipedia.org/wiki/Logistic_function), given by 我们将使用的 S 型函数被称作 [逻辑函数](https://en.wikipedia.org/wiki/Logistic_function)，它的公式是：

$logistic(x) = \cfrac{1}{1+e^{-\beta x}}$

where $\beta$ is the slope factor. However, computing the exponential function in hardware is quite challenging and expensive. We'll approximate this as the step function. 其中，$\beta$ 是坡度因子。然而，在硬件中计算指数是非常困难并且昂贵的。

$step(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             1  & \text{if } x \gt 0
       \end{cases}$

The second function, the ReLU, is given by a similar formula. 第二个函数是 ReLU，它也是被类似的公式定义的：

$relu(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             x  & \text{if } x \gt 0
       \end{cases}$

Implement these two functions below. You can specify a fixed-point literal like `-3.14.F(8.BP)`. 在下面实现这两个函数。你可以自定义一个固定小数的文字型数据，就像`-3.14.F(8.BP)`。

In [ ]:
val Step: FixedPoint => FixedPoint = ???
val ReLU: FixedPoint => FixedPoint = ???

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution 解答</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val Step: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), 1.F(8.BP))
val ReLU: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), x)
</pre></article></div></section></div>

Finally, let's create a tester that checks the correctness of our Neuron. With the step activation function, neurons may be used as logic gate approximators. Proper selection of weights and bias can perform binary functions. We'll test our neuron using AND logic. Complete the following tester to check our neuron with the step function. 最后，让我们创建一个测试我们神经元的模块。因为带有阶跃激活函数，我们的神经元可能被用作逻辑门近似器。我们将使用与逻辑测试我们的神经元。完成以下的测试模块以使用阶跃函数检查我们的神经元。

Note that since the circuit is purely combinational, the `reset(5)` and `step(1)` calls are not necessary. 注意：因为这个电路是存粹的组合电路，所以诸如 `reset(5)` 和 `step(1)` 这样的语法就不是必须的。

In [ ]:
// test our Neuron 测试我们的神经元
Driver(() => new Neuron(2, Step)) {
  c => new PeekPokeTester(c) {
    
    val inputs = Seq(Seq(-1, -1), Seq(-1, 1), Seq(1, -1), Seq(1, 1))
    
    // make this a sequence of two values 使得权重是两个值的序列
    val weights = ???

    // push data through our Neuron and check the result (AND gate) 把数据输入到我们的神经元中并且检查结果 (与门)
    reset(5)
    for (i <- inputs) {
      pokeFixedPoint(c.io.in(0), i(0))
      pokeFixedPoint(c.io.in(1), i(1))
      pokeFixedPoint(c.io.weights(0), weights(0))
      pokeFixedPoint(c.io.weights(1), weights(1))
      expectFixedPoint(c.io.out, if (i(0) + i(1) > 0) 1 else 0, "ERROR")
      step(1)
    }
  }
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>Solution 解答</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val weights  = Seq(1.0, 1.0)
</pre></article></div></section></div>

---
# You're done! 恭喜你，完成了本节内容的学习！

[Return to the top. 回到顶层。](#top)